In [ ]:
import cv2,cvzone
from cvzone.ColorModule import ColorFinder

cap = cv2.VideoCapture('./test.webm')
myColorFinder = ColorFinder(trackBar=False)
hsvVals = {'hmin': 0, 'smin': 140, 'vmin': 148, 'hmax': 7, 'smax': 255, 'vmax': 255}

posList = []

while True:
    ret, img = cap.read()
    # img = cv2.imread('ball.png')

    imgColor, mask = myColorFinder.update(img, hsvVals)
    imgContours, contours = cvzone.findContours(img,mask,minArea=50)

    if contours:
        posList.append(contours[0]['center'])
    
    for i,pos in enumerate(posList):
        cv2.circle(imgContours,pos,5,(0,255,0),cv2.FILLED)
        if i==0:
            continue
        cv2.line(imgContours,pos,posList[i-1],(255,0,0),2)

    cv2.imshow("Color Filtered Image", imgContours)

    # CRITICAL: Use waitKey(1) to allow the GUI to update
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


: 

In [3]:
import cv2, zmq, numpy as np, time, threading, queue, traceback, sys
from collections import deque, defaultdict

# ---------- Config ----------
ZMQ_ADDR = "tcp://localhost:5555"
SUB_TOPICS = [b"kreo1", b"kreo2"]
FPS_WINDOW = 1.0        # seconds for fps moving window
DISPLAY_FPS = 20
VISUALIZE = True     # show tiled view window

# color thresholds (you gave these)
orange_hsvVals = {'hmin': 0, 'smin': 100, 'vmin': 100, 'hmax': 25, 'smax': 255, 'vmax': 255}
purple_hsvVals = {'hmin': 149, 'smin': 69, 'vmin': 82, 'hmax': 177, 'smax': 229, 'vmax': 252}

# detector parameters
MIN_AREA = 100    # min contour area to accept (tune if needed)
MAX_AREA = 20000  # max area (avoid very large blobs)
CIRCULARITY_MIN = 0.25  # min circularity to accept (lower because paper balls can deform)
ASPECT_RATIO_MAX = 2.0   # reject extremely elongated blobs
MAX_DETECTIONS_PER_CAM = 12  # safety limi


# ---------- Helpers ----------
def fmt_ts(ts):
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ts)) + f".{int((ts%1)*1000):03d}"

def recv_latest(sub):
    msg = None
    while True:
        try:
            msg = sub.recv_multipart(flags=zmq.NOBLOCK)
        except zmq.Again:
            break
    return msg

def update_fps(camera, cam_ts):
    dq = fps_windows[camera]
    dq.append(cam_ts)
    # pop older than window
    while dq and (cam_ts - dq[0]) > FPS_WINDOW:
        dq.popleft()
    fps = len(dq) / FPS_WINDOW
    return fps

# ---------- Color masking helpers ----------
def hsv_mask_from_vals(bgr_img, hsvVals):
    """Return binary mask from HSV thresholds dict."""
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)
    lower = np.array([hsvVals['hmin'], hsvVals['smin'], hsvVals['vmin']], dtype=np.uint8)
    upper = np.array([hsvVals['hmax'], hsvVals['smax'], hsvVals['vmax']], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower, upper)
    # small blur to reduce speckle
    mask = cv2.medianBlur(mask, 5)
    return mask

def postprocess_mask(mask):
    """Morphological clean-up."""
    # open then close
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    m = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kernel, iterations=1)
    return m

def find_candidate_contours(mask, min_area=MIN_AREA, max_area=MAX_AREA):
    """Return list of contours filtered by area and shape heuristics."""
    if mask is None:
        return []
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < min_area or area > max_area:
            continue
        perim = cv2.arcLength(c, True)
        if perim <= 0:
            continue
        circularity = 4 * np.pi * area / (perim * perim)
        x,y,w,h = cv2.boundingRect(c)
        aspect = float(w)/float(h) if h>0 else 0.0
        # accept if roughly circular-ish or moderate area even if circularity low
        if circularity >= CIRCULARITY_MIN or (0.5*min(w,h) > 5 and area > (min_area*2)):
            if aspect <= ASPECT_RATIO_MAX:
                candidates.append({
                    "contour": c,
                    "area": area,
                    "perimeter": perim,
                    "circularity": circularity,
                    "bbox": (int(x),int(y),int(w),int(h))
                })
    # sort by area desc
    candidates.sort(key=lambda d: d["area"], reverse=True)
    return candidates


# ---------- Detector thread (per-camera) ----------
class BallDetectorThread(threading.Thread):
    def __init__(self, cam_name, frame_queue, detect_cache, lock):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.detect_cache = detect_cache
        self.lock = lock
        self.stop_flag = False

    def run(self):
        print(f"[{self.cam_name}] Detector thread started.")
        while not self.stop_flag:
            try:
                frame,cam_ts = self.frame_queue.get(timeout=0.1)
            except queue.Empty:
                continue
            try:
                # build masks for both colors
                mask_orange = hsv_mask_from_vals(frame, orange_hsvVals)
                mask_purple = hsv_mask_from_vals(frame, purple_hsvVals)

                # combine & clean
                combined_mask = cv2.bitwise_or(mask_orange, mask_purple)
                combined_mask = postprocess_mask(combined_mask)

                # find contours
                candidates = find_candidate_contours(combined_mask)

                detections = []
                for cand in candidates[:MAX_DETECTIONS_PER_CAM]:
                    c = cand["contour"]
                    x,y,w,h = cand["bbox"]
                    area = cand["area"]
                    circ = cand["circularity"]

                    # centroid
                    M = cv2.moments(c)
                    if M["m00"] != 0:
                        cx = int(M["m10"]/M["m00"]); cy = int(M["m01"]/M["m00"])
                    else:
                        cx = x + w//2; cy = y + h//2

                    # classify color by sampling the masks inside bbox
                    s_orange = int(np.count_nonzero(mask_orange[y:y+h, x:x+w])) if mask_orange is not None else 0
                    s_purple = int(np.count_nonzero(mask_purple[y:y+h, x:x+w])) if mask_purple is not None else 0

                    # small area leads to ambiguity; prefer stronger mask
                    color = "unknown"
                    if s_orange > s_purple and s_orange > 0:
                        color = "orange"
                    elif s_purple > s_orange and s_purple > 0:
                        color = "purple"
                    else:
                        # fallback: mean hue in bbox
                        try:
                            hsv_roi = cv2.cvtColor(frame[y:y+h, x:x+w], cv2.COLOR_BGR2HSV)
                            mean_h = int(np.mean(hsv_roi[:,:,0]))
                            if orange_hsvVals['hmin'] <= mean_h <= orange_hsvVals['hmax']:
                                color = "orange"
                            elif purple_hsvVals['hmin'] <= mean_h <= purple_hsvVals['hmax']:
                                color = "purple"
                        except Exception:
                            color = "unknown"

                    det = {
                        "bbox": (int(x),int(y),int(w),int(h)),
                        "centroid": (int(cx),int(cy)),
                        "area": float(area),
                        "circularity": float(circ),
                        "color": color,
                        "ts": float(cam_ts),
                        "det_time": time.time()
                    }
                    detections.append(det)

                with self.lock:
                    self.detect_cache[self.cam_name] = detections

            except Exception as e:
                print(f"[ERROR-{self.cam_name}] detection exception:", e)
                traceback.print_exc()

        print(f"[{self.cam_name}] BallDetectorThread stopped")

    def stop(self):
        self.stop_flag = True

# ---------- ZMQ subscriber ----------
ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.connect(ZMQ_ADDR)
sub.setsockopt(zmq.RCVHWM, 1)
sub.setsockopt(zmq.CONFLATE, 1)  # keep only last message
sub.setsockopt(zmq.LINGER, 0)

# ---- ACTIVE FLUSH ----
flushed = 0
while True:
    try:
        sub.recv_multipart(flags=zmq.NOBLOCK)
        flushed += 1
    except zmq.Again:
        break
if flushed > 0:
    print(f"[Subscriber] Flushed {flushed} stale messages.")
for t in SUB_TOPICS:
    sub.setsockopt(zmq.SUBSCRIBE, t)


# per-camera structures
frames = {}
fps_windows = defaultdict(lambda: deque())
frame_queues = {t.decode(): queue.Queue(maxsize=1) for t in SUB_TOPICS}
detect_cache = {}        # cam -> detection dict
detect_lock = threading.Lock()
det_threads = {}

for t in SUB_TOPICS:
    cam_name = t.decode()
    dt = BallDetectorThread(cam_name, frame_queues[cam_name], detect_cache, detect_lock)
    dt.start()
    det_threads[cam_name] = dt


print("[Subscriber] connected, waiting for frames... (Press ESC to exit)")

last_show = time.time()
try:
    while True:
        parts = recv_latest(sub)
        if parts is None:
            continue

        # unpack message
        topic = parts[0]; cam = topic.decode()
        if len(parts) >= 3:
            ts_part = parts[1]; jpg_part = parts[2]
        else:
            ts_part = None; jpg_part = parts[1]

        recv_time = time.time()
        try:
            cam_ts = float(ts_part.decode()) if ts_part else recv_time
        except:
            cam_ts = recv_time

        img = cv2.imdecode(np.frombuffer(jpg_part, np.uint8), cv2.IMREAD_COLOR)
        if img is None:
            continue

        fps = update_fps(cam, cam_ts)
        frames[cam] = {"img": img, "cam_ts": cam_ts, "fps": fps}

        # push latest frame to detector queue (maxsize=1)
        fq = frame_queues[cam]
        try:
            fq.get_nowait()
        except queue.Empty:
            pass
        try:
            fq.put_nowait((img.copy(), cam_ts))
        except queue.Full:
            pass

        # build tiled view if both cams available
        if VISUALIZE and all(k in frames for k in [t.decode() for t in SUB_TOPICS]):
            cams = [t.decode() for t in SUB_TOPICS]
            left = frames[cams[0]]; right = frames[cams[1]]
            drift_ms = abs(left["cam_ts"] - right["cam_ts"]) * 1000.0

            def overlay(frame_info, cam_name):
                im = frame_info["img"].copy()
                y = 20
                cv2.putText(im, f"{cam_name}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,32,20), 2)
                cv2.putText(im, f"FPS: {frame_info['fps']:.1f}", (10, y+26), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (14,117,5), 2)
                cv2.putText(im, f"cam_ts: {fmt_ts(frame_info['cam_ts'])}", (10, y+52), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (5,12,117), 1)

                # draw ALL cached detections for this camera
                with detect_lock:
                    dets = detect_cache.get(cam_name, [])
                    for i, d in enumerate(dets):
                        x,y,w,h = d["bbox"]
                        cx,cy = d["centroid"]
                        color = d.get("color", "unknown")
                        box_color = (0,200,200)  # default
                        if color == "orange":
                            box_color = (0,200,255)  # orange-ish
                        elif color == "purple":
                            box_color = (200,0,200)  # purple-ish
                        # draw bounding box and centroid
                        cv2.rectangle(im, (x,y), (x+w, y+h), box_color, 2)
                        cv2.circle(im, (cx,cy), 4, (0,0,255), -1)
                        cv2.putText(im, f"{color}:{i}", (x, y-6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)
                return im

            left_im = overlay(left, cams[0])
            right_im = overlay(right, cams[1])

            # tile horizontally
            h = max(left_im.shape[0], right_im.shape[0])
            right_resized = cv2.resize(right_im, (left_im.shape[1], h))
            tile = np.hstack([left_im, right_resized])

            # overlays
            cv2.putText(tile, f"Drift: {drift_ms:.1f} ms", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
            cv2.putText(tile, f"Host now: {fmt_ts(time.time())}", (10, 44), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

            # display throttled
            if VISUALIZE and (time.time() - last_show) > (1.0/DISPLAY_FPS):
                last_show = time.time()
                cv2.imshow("Both Cameras (tiled)", tile)
        elif not VISUALIZE:
            with detect_lock:
                parts_status = []
                for c in [t.decode() for t in SUB_TOPICS]:
                    dets = detect_cache.get(c, [])
                    if dets:
                        counts = { "orange":0, "purple":0, "unknown":0 }
                        for d in dets:
                            counts[d.get("color","unknown")] = counts.get(d.get("color","unknown"),0) + 1
                        parts_status.append(f"{c}: Orange: {counts['orange']} Purple: {counts['purple']}")
                    else:
                        parts_status.append(f"{c}:NoBall")
                sys.stdout.write("\r" + " | ".join(parts_status) + " " * 20)
                sys.stdout.flush()

        if cv2.waitKey(1) & 0xFF == 27:
            break

except KeyboardInterrupt:
    pass
finally:
    # stop threads
    for d in det_threads.values():
        d.stop()
    # allow threads to exit
    time.sleep(0.1)
    cv2.destroyAllWindows()
    sub.close()
    ctx.term()
    print("Exit clean.")

[kreo1] Detector thread started.
[kreo2] Detector thread started.
[Subscriber] connected, waiting for frames... (Press ESC to exit)
[kreo2] BallDetectorThread stopped
[kreo1] BallDetectorThread stopped
Exit clean.


In [ ]:
import cv2, zmq, numpy as np, time, threading, queue, traceback, sys, os, csv
from collections import deque, defaultdict

# ---------- Config ----------
ZMQ_ADDR = "tcp://localhost:5555"
SUB_TOPICS = [b"kreo1", b"kreo2"]
FPS_WINDOW = 1.0        # seconds for fps moving window
DISPLAY_FPS = 60        # Limit display refresh to save CPU for detection
VISUALIZE = False        # Set to False on headless systems to save even more FPS

# OPTIMIZATION: Downscale frame for detection (0.5 = 360p, 4x faster than 720p)
DETECTION_SCALE = 0.5 

# LOGGING PATH (Updated as per request)
LOG_DIR = "../../data/ball_detection_logs"
LOG_FILENAME = f"{LOG_DIR}/data_log_{int(time.time())}.csv"

# PER-CAMERA HSV CONFIG
# Using specific values for kreo1 and kreo2 as requested.
HSV_CONFIG = {
    "kreo1": {
        "orange": {'hmin': 0, 'smin': 120, 'vmin': 175, 'hmax': 12, 'smax': 255, 'vmax': 255},
    },
    "kreo2": {
        "orange": {'hmin': 0, 'smin': 150, 'vmin': 150, 'hmax': 12, 'smax': 255, 'vmax': 255}
    }
}

# Fallback for unknown cameras
DEFAULT_HSV = {'hmin': 0, 'smin': 100, 'vmin': 100, 'hmax': 25, 'smax': 255, 'vmax': 255}

# Detector parameters (Adjusted for Scale in main logic)
BASE_MIN_AREA = 100    
BASE_MAX_AREA = 20000  
CIRCULARITY_MIN = 0.5
ASPECT_RATIO_MIN = 0.6 
ASPECT_RATIO_MAX = 1.6   
MAX_DETECTIONS_PER_CAM = 5 

# ---------- Logging Setup ----------
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

# We use a thread-safe queue for logging to avoid blocking detection threads with file I/O
log_queue = queue.Queue()

def logger_worker():
    """Writes logs to CSV from a queue."""
    try:
        with open(LOG_FILENAME, 'w', newline='') as f:
            writer = csv.writer(f)
            # Header
            writer.writerow(["timestamp", "camera", "status", "color", "x", "y", "w", "h", "area"])
            
            while True:
                entry = log_queue.get()
                if entry is None: # Poison pill
                    break
                writer.writerow(entry)
                log_queue.task_done()
    except Exception as e:
        print(f"[LOGGER ERROR] {e}")

# Start logger thread
log_thread = threading.Thread(target=logger_worker, daemon=True)
log_thread.start()

def log_frame_result(ts, cam, status, det_data=None):
    """
    Helper to push data to log queue.
    status: DETECTED, NOT_DETECTED, SKIPPED_TIMEOUT
    """
    if det_data:
        x, y, w, h = det_data['bbox']
        color = det_data.get('color', 'unknown')
        area = det_data.get('area', 0)
        log_queue.put([f"{ts:.3f}", cam, status, color, x, y, w, h, int(area)])
    else:
        # No detection data (NOT_DETECTED or SKIPPED)
        log_queue.put([f"{ts:.3f}", cam, status, "N/A", "", "", "", "", ""])


# ---------- Helpers ----------
def fmt_ts(ts):
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ts)) + f".{int((ts%1)*1000):03d}"

def recv_latest(sub):
    msg = None
    while True:
        try:
            msg = sub.recv_multipart(flags=zmq.NOBLOCK)
        except zmq.Again:
            break
    return msg

def update_fps(camera, cam_ts):
    dq = fps_windows[camera]
    dq.append(cam_ts)
    # pop older than window
    while dq and (cam_ts - dq[0]) > FPS_WINDOW:
        dq.popleft()
    fps = len(dq) / FPS_WINDOW
    return fps

# ---------- Color masking helpers ----------
def get_orange_mask(bgr_img, hsv_dict):
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)
    lower_o = np.array([hsv_dict['hmin'], hsv_dict['smin'], hsv_dict['vmin']], dtype=np.uint8)
    upper_o = np.array([hsv_dict['hmax'], hsv_dict['smax'], hsv_dict['vmax']], dtype=np.uint8)
    mask_o = cv2.inRange(hsv, lower_o, upper_o)
    
    mask_o = cv2.GaussianBlur(mask_o, (5, 5), 0)
    return mask_o

def find_candidate_contours(mask, min_area, max_area):
    if mask is None:
        return []
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    candidates = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < min_area or area > max_area:
            continue
            
        # Bounding rect
        x,y,w,h = cv2.boundingRect(c)
        aspect = float(w)/float(h) if h > 0 else 0
        if ASPECT_RATIO_MIN>aspect or aspect > ASPECT_RATIO_MAX:
            continue
            
        perim = cv2.arcLength(c, True)
        if perim == 0: continue
        
        circularity = 4 * np.pi * area / (perim * perim)
        
        # Accept logic
        if circularity >= CIRCULARITY_MIN:
            candidates.append({
                "contour": c,
                "area": area,
                "circularity": circularity,
                "bbox": (x,y,w,h)
            })
            
    # sort by area desc
    candidates.sort(key=lambda d: d["area"], reverse=True)
    return candidates


# ---------- Detector thread (per-camera) ----------
class BallDetectorThread(threading.Thread):
    def __init__(self, cam_name, frame_queue, detect_cache, lock):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.detect_cache = detect_cache
        self.lock = lock
        self.stop_flag = False
        
        # Pre-calculate scaled area thresholds
        self.min_area_scaled = BASE_MIN_AREA * (DETECTION_SCALE * DETECTION_SCALE)
        self.max_area_scaled = BASE_MAX_AREA * (DETECTION_SCALE * DETECTION_SCALE)

        # Get camera specific HSV
        cam_config = HSV_CONFIG.get(cam_name, {})
        self.hsv_vals = cam_config.get("orange", DEFAULT_HSV)
        print(f"[{self.cam_name}] Configured with HSV: {self.hsv_vals}")

    def run(self):
        print(f"[{self.cam_name}] Detector thread started. Scale: {DETECTION_SCALE}")
        while not self.stop_flag:
            try:
                # Wait for frame
                frame_orig, cam_ts = self.frame_queue.get(timeout=0.1)
            except queue.Empty:
                continue
            
            detection_found = False
            
            try:
                # 1. Downscale for speed
                if DETECTION_SCALE != 1.0:
                    frame_proc = cv2.resize(frame_orig, None, fx=DETECTION_SCALE, fy=DETECTION_SCALE, interpolation=cv2.INTER_NEAREST)
                else:
                    frame_proc = frame_orig

                # 2. Get Mask (Only Orange as requested)
                mask_o = get_orange_mask(frame_proc, self.hsv_vals)

                # 3. Find Contours
                candidates = find_candidate_contours(mask_o, self.min_area_scaled, self.max_area_scaled)

                detections = []
                
                # 4. Process Top Candidates
                for cand in candidates[:MAX_DETECTIONS_PER_CAM]:
                    x_s, y_s, w_s, h_s = cand["bbox"] # scaled coords
                    
                    # Scale coords back to original resolution
                    scale_inv = 1.0 / DETECTION_SCALE
                    x = int(x_s * scale_inv)
                    y = int(y_s * scale_inv)
                    w = int(w_s * scale_inv)
                    h = int(h_s * scale_inv)
                    area_orig = cand["area"] * (scale_inv * scale_inv)

                    cx = x + w//2
                    cy = y + h//2

                    det = {
                        "bbox": (x,y,w,h),
                        "centroid": (cx,cy),
                        "area": area_orig,
                        "color": "orange", # We are only checking orange
                        "ts": cam_ts
                    }
                    detections.append(det)
                    
                    # Log POSITIVE detection
                    log_frame_result(cam_ts, self.cam_name, "DETECTED", det)
                    detection_found = True
                
                # Update shared cache for visualization
                with self.lock:
                    self.detect_cache[self.cam_name] = detections

            except Exception as e:
                print(f"[ERROR-{self.cam_name}] detection exception:", e)
                traceback.print_exc()
            
            # Log NEGATIVE detection (if nothing found in this frame)
            if not detection_found:
                log_frame_result(cam_ts, self.cam_name, "NOT_DETECTED")

        print(f"[{self.cam_name}] BallDetectorThread stopped")

    def stop(self):
        self.stop_flag = True

# ---------- ZMQ subscriber ----------
ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.connect(ZMQ_ADDR)
sub.setsockopt(zmq.RCVHWM, 10)
# sub.setsockopt(zmq.CONFLATE, 1) 
sub.setsockopt(zmq.LINGER, 0)
for t in SUB_TOPICS:
    sub.setsockopt(zmq.SUBSCRIBE, t)

# ---- ACTIVE FLUSH ----
flushed = 0
while True:
    try:
        sub.recv_multipart(flags=zmq.NOBLOCK)
        flushed += 1
    except zmq.Again:
        break
if flushed > 0:
    print(f"[Subscriber] Flushed {flushed} stale messages.")


# per-camera structures
frames = {}
fps_windows = defaultdict(lambda: deque())
frame_queues = {t.decode(): queue.Queue(maxsize=1) for t in SUB_TOPICS}
detect_cache = {}        
detect_lock = threading.Lock()
det_threads = {}

# Start threads
for t in SUB_TOPICS:
    cam_name = t.decode()
    # Verify we have config for this camera
    if cam_name not in HSV_CONFIG:
        print(f"[WARNING] No specific HSV config found for {cam_name}, using defaults.")
    
    dt = BallDetectorThread(cam_name, frame_queues[cam_name], detect_cache, detect_lock)
    dt.start()
    det_threads[cam_name] = dt


print(f"[Subscriber] Connected. Logging to: {LOG_FILENAME}")
print(f"[Subscriber] Press ESC to exit.")

last_show = time.time()

try:
    while True:
        latest_msgs = {}
        
        while True:
            try:
                parts = sub.recv_multipart(flags=zmq.NOBLOCK)
                topic = parts[0]
                latest_msgs[topic] = parts # Overwrite with newer if multiple exist
            except zmq.Again:
                break
        
        # Process the unique latest frame for each camera found
        if not latest_msgs:
            # No data, brief sleep to yield CPU
            time.sleep(0.001)
        else:
            for topic, parts in latest_msgs.items():
                cam = topic.decode()
                ts_part = parts[1] if len(parts) >= 3 else None
                jpg_part = parts[2] if len(parts) >= 3 else parts[1]
                
                recv_time = time.time()
                try:
                    cam_ts = float(ts_part.decode()) if ts_part else recv_time
                except:
                    cam_ts = recv_time

                # Decode
                img = cv2.imdecode(np.frombuffer(jpg_part, np.uint8), cv2.IMREAD_COLOR)
                if img is None: continue

                fps = update_fps(cam, cam_ts)
                frames[cam] = {"img": img, "cam_ts": cam_ts, "fps": fps}

                # Push to detector
                fq = frame_queues[cam]
                try:
                    fq.put_nowait((img, cam_ts))
                except queue.Full:
                    # Drop old, put new (LIFO)
                    try:
                        _dropped_frame, _dropped_ts = fq.get_nowait()
                        log_frame_result(_dropped_ts, cam, "SKIPPED_TIMEOUT")
                        fq.put_nowait((img, cam_ts))
                    except queue.Empty:
                        pass

        # Visualization (Throttled)
        if VISUALIZE and (time.time() - last_show) > (1.0/DISPLAY_FPS):
            if all(k in frames for k in [t.decode() for t in SUB_TOPICS]):
                cams = [t.decode() for t in SUB_TOPICS]
                left, right = frames[cams[0]], frames[cams[1]]
                
                def overlay(frame_info, cam_name):
                    im = frame_info["img"].copy()
                    cv2.putText(im, f"{cam_name} FPS: {frame_info['fps']:.1f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                    with detect_lock:
                        for d in detect_cache.get(cam_name, []):
                            x,y,w,h = d["bbox"]
                            cv2.rectangle(im, (x,y), (x+w, y+h), (0,165,255), 2)
                            cv2.circle(im, d["centroid"], 5, (0,0,255), -1)
                    return im

                l_im = overlay(left, cams[0])
                r_im = overlay(right, cams[1])
                
                h = min(l_im.shape[0], r_im.shape[0])
                if l_im.shape[0] != h: l_im = cv2.resize(l_im, (int(l_im.shape[1]*h/l_im.shape[0]), h))
                if r_im.shape[0] != h: r_im = cv2.resize(r_im, (int(r_im.shape[1]*h/r_im.shape[0]), h))
                
                cv2.imshow("Stereo View", np.hstack([l_im, r_im]))
                last_show = time.time()
        
        elif not VISUALIZE:
            sys.stdout.write(f"\rFPS: {frames.get('kreo1',{}).get('fps',0):.1f} | {frames.get('kreo2',{}).get('fps',0):.1f}")
            sys.stdout.flush()

        if cv2.waitKey(1) & 0xFF == 27:
            break

except KeyboardInterrupt:
    pass
finally:
    for d in det_threads.values(): d.stop()
    log_queue.put(None)
    log_thread.join()
    cv2.destroyAllWindows()
    sub.close()
    ctx.term()
    print("\nLog saved to:", LOG_FILENAME)

[kreo1] Configured with HSV: {'hmin': 0, 'smin': 120, 'vmin': 175, 'hmax': 12, 'smax': 255, 'vmax': 255}
[kreo1] Detector thread started. Scale: 0.5
[kreo2] Configured with HSV: {'hmin': 0, 'smin': 150, 'vmin': 150, 'hmax': 12, 'smax': 255, 'vmax': 255}
[kreo2] Detector thread started. Scale: 0.5
[Subscriber] Connected. Logging to: ../../data/ball_detection_logs/data_log_1763878277.csv
[Subscriber] Press ESC to exit.
FPS: 40.0 | 40.0
Log saved to: ../../data/ball_detection_logs/data_log_1763878277.csv


[kreo1] BallDetectorThread stopped
[kreo2] BallDetectorThread stopped
